# How to Develop LSTM Models for Time Series Forecasting
https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/

In [2]:
import os
os.chdir('..')

In [9]:
import workload

df = workload.get('compute1')
df = df.resample('s').interpolate().resample('90s').asfreq()
df = df.dropna()

#create numpy.ndarray
df1 = df['mem']
df_arr = df1.values
df_arr

array([17.21364499, 32.88770053, 37.90998217, 58.39795009, 75.26737968,
       85.29411765, 85.29411765, 85.29411765, 85.29411765, 85.29411765,
       85.29411765, 85.29411765, 85.31568052, 85.29411765, 85.29411765,
       85.3097148 , 85.30748663, 85.36096257, 85.36096257, 85.36096257,
       85.32308378, 85.36096257, 85.36096257, 85.36096257, 85.36096257,
       75.64688632, 64.63903743, 53.74331551, 33.65318268, 22.45989305,
       11.96524064, 22.59358289, 32.88770053, 32.90106952, 42.78074866,
       52.74064171])

In [10]:
# univariate data preparation
from numpy import array
 
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)
 
split_sequence(df_arr, 3)

(array([[17.21364499, 32.88770053, 37.90998217],
        [32.88770053, 37.90998217, 58.39795009],
        [37.90998217, 58.39795009, 75.26737968],
        [58.39795009, 75.26737968, 85.29411765],
        [75.26737968, 85.29411765, 85.29411765],
        [85.29411765, 85.29411765, 85.29411765],
        [85.29411765, 85.29411765, 85.29411765],
        [85.29411765, 85.29411765, 85.29411765],
        [85.29411765, 85.29411765, 85.29411765],
        [85.29411765, 85.29411765, 85.29411765],
        [85.29411765, 85.29411765, 85.31568052],
        [85.29411765, 85.31568052, 85.29411765],
        [85.31568052, 85.29411765, 85.29411765],
        [85.29411765, 85.29411765, 85.3097148 ],
        [85.29411765, 85.3097148 , 85.30748663],
        [85.3097148 , 85.30748663, 85.36096257],
        [85.30748663, 85.36096257, 85.36096257],
        [85.36096257, 85.36096257, 85.36096257],
        [85.36096257, 85.36096257, 85.32308378],
        [85.36096257, 85.32308378, 85.36096257],
        [85.32308378

In [11]:
# SAMPLES

# define input sequence
raw_seq = df_arr

# choose a number of time steps
n_steps = 3
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

[17.21364499 32.88770053 37.90998217] 58.39795008912656
[32.88770053 37.90998217 58.39795009] 75.26737967914438
[37.90998217 58.39795009 75.26737968] 85.29411764705883
[58.39795009 75.26737968 85.29411765] 85.29411764705883
[75.26737968 85.29411765 85.29411765] 85.29411764705883
[85.29411765 85.29411765 85.29411765] 85.29411764705883
[85.29411765 85.29411765 85.29411765] 85.29411764705883
[85.29411765 85.29411765 85.29411765] 85.29411764705883
[85.29411765 85.29411765 85.29411765] 85.29411764705883
[85.29411765 85.29411765 85.29411765] 85.31568052440917
[85.29411765 85.29411765 85.31568052] 85.29411764705883
[85.29411765 85.31568052 85.29411765] 85.29411764705883
[85.31568052 85.29411765 85.29411765] 85.30971479500892
[85.29411765 85.29411765 85.3097148 ] 85.30748663101605
[85.29411765 85.3097148  85.30748663] 85.36096256684492
[85.3097148  85.30748663 85.36096257] 85.36096256684492
[85.30748663 85.36096257 85.36096257] 85.36096256684492
[85.36096257 85.36096257 85.36096257] 85.3230837

In [12]:
# univariate lstm example
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
 
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	print(array(X), array(y))
	return array(X), array(y)
 
# define input sequence
raw_seq = df_arr
# choose a number of time steps
n_steps = 10
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=0)
# demonstrate prediction
x_input = array(df_arr[-n_steps:])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(x_input)
print(yhat)

[[17.21364499 32.88770053 37.90998217 58.39795009 75.26737968 85.29411765
  85.29411765 85.29411765 85.29411765 85.29411765]
 [32.88770053 37.90998217 58.39795009 75.26737968 85.29411765 85.29411765
  85.29411765 85.29411765 85.29411765 85.29411765]
 [37.90998217 58.39795009 75.26737968 85.29411765 85.29411765 85.29411765
  85.29411765 85.29411765 85.29411765 85.29411765]
 [58.39795009 75.26737968 85.29411765 85.29411765 85.29411765 85.29411765
  85.29411765 85.29411765 85.29411765 85.31568052]
 [75.26737968 85.29411765 85.29411765 85.29411765 85.29411765 85.29411765
  85.29411765 85.29411765 85.31568052 85.29411765]
 [85.29411765 85.29411765 85.29411765 85.29411765 85.29411765 85.29411765
  85.29411765 85.31568052 85.29411765 85.29411765]
 [85.29411765 85.29411765 85.29411765 85.29411765 85.29411765 85.29411765
  85.31568052 85.29411765 85.29411765 85.3097148 ]
 [85.29411765 85.29411765 85.29411765 85.29411765 85.29411765 85.31568052
  85.29411765 85.29411765 85.3097148  85.30748663]
